# 🕵️‍♂️ Project Nexus: Client 360 Assistant
**Module:** Agentic AI Day 2 (Tools & Mechanics)  
**Objective:** Build an AI Agent that can retrieve client info and schedule meetings using structured inputs and a manual ReAct loop.

---

### 📋 Requirements Checklist
1.  **Tool A (`get_client_details`):** Retrieve data from a mock DB. Handle "Client Not Found" errors gracefully.
2.  **Tool B (`schedule_meeting`):** Use Pydantic to ensure `duration` is an integer.
3.  **The Loop:** Manually execute the `Human` -> `AI` -> `Tool` -> `AI` cycle.
4.  **Binding:** Explicitly bind tools to `gpt-4o`.

In [1]:
# CELL 1: SETUP & API KEY
import os
import getpass
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, StructuredTool
#from langchain_core.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

# Initialize the Brain
llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("✅ Environment Setup Complete")

✅ Environment Setup Complete


In [2]:
# CELL 2: MOCK DATABASE
# This simulates our internal CRM
CLIENT_DB = {
    "Acme Corp": {"industry": "Retail", "contact": "Alice Smith"},
    "Globex": {"industry": "Tech", "contact": "Bob Jones"},
    "Soylent Corp": {"industry": "Food", "contact": "Carol White"}
}
print("✅ Database Loaded")

✅ Database Loaded


### 🛠️ Step 1: Define the Tools
Here we define our two tools. 
* `get_client_details` uses standard python logic.
* `schedule_meeting` uses a **Pydantic Model** to enforce strict types (Integer for duration).

In [3]:
# CELL 3: TOOL DEFINITIONS

# --- TOOL A: RETRIEVAL ---
@tool
def get_client_details(client_name: str) -> str:
    """Retrieves industry and key contact for a client.
    Returns an error string if client is not found."""
    
    # Simulate Case Insensitive Search
    for key, value in CLIENT_DB.items():
        if key.lower() == client_name.lower():
            return f"Client: {key} | Industry: {value['industry']} | Contact: {value['contact']}"
            
    # Graceful Failure (Return string, don't crash)
    return f"Error: Client '{client_name}' not found in the database."

# --- TOOL B: SCHEDULER (With Pydantic) ---
class ScheduleInput(BaseModel):
    client_name: str = Field(description="Name of the client")
    date: str = Field(description="Date of the meeting (e.g., 'Oct 20th')")
    duration_minutes: int = Field(description="Duration in minutes (must be an integer)")

def schedule_meeting_func(client_name: str, date: str, duration_minutes: int) -> str:
    """Schedules a meeting in the calendar."""
    return f"SUCCESS: Meeting scheduled with {client_name} on {date} for {duration_minutes} minutes."

# Wrap function with Pydantic Schema
schedule_tool = StructuredTool.from_function(
    func=schedule_meeting_func,
    name="schedule_meeting",
    description="Schedule a meeting with a client.",
    args_schema=ScheduleInput
)

print("✅ Tools Defined")

✅ Tools Defined


### 🔗 Step 2: Binding
We bind the tools to the LLM so it knows they exist.

In [4]:
# CELL 4: BINDING
tools = [get_client_details, schedule_tool]
llm_with_tools = llm.bind_tools(tools)

# Create a map for easy lookup during execution
tool_map = {
    "get_client_details": get_client_details,
    "schedule_meeting": schedule_tool
}
print("✅ Tools Bound to LLM")

✅ Tools Bound to LLM


### ⚙️ Step 3: The Manual Execution Loop
This is the core logic. We manually handle the `ToolCalls` and message history.

In [5]:
# CELL 5: THE LOOP
def run_agent(user_query: str):
    print(f"\n🚀 STARTING AGENT FOR: '{user_query}'")
    print("-" * 50)
    
    # Initialize State
    messages = [HumanMessage(content=user_query)]
    
    # Limit loop to prevent infinite runs (Safety Mechanism)
    for i in range(5): 
        # A. Invoke Brain
        response = llm_with_tools.invoke(messages)
        messages.append(response) # Update Memory
        
        # B. Check for Tool Calls
        if response.tool_calls:
            print(f"🤖 AI Decided to call: {len(response.tool_calls)} tools")
            
            # C. Execute Tools
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                tool_args = tool_call["args"]
                tool_id = tool_call["id"]
                
                print(f"   👉 Invoking '{tool_name}' with args: {tool_args}")
                
                # Execute the actual Python function
                selected_tool = tool_map[tool_name]
                try:
                    tool_result = selected_tool.invoke(tool_args)
                except Exception as e:
                    tool_result = f"Error executing tool: {e}"
                
                print(f"   ✅ Result: {tool_result}")
                
                # Create Tool Message and add to history
                tool_msg = ToolMessage(content=str(tool_result), tool_call_id=tool_id)
                messages.append(tool_msg)
        else:
            # D. No more tools needed -> Final Answer
            print("-" * 50)
            print(f"🏁 FINAL ANSWER: {response.content}")
            return response.content
            
    print("❌ Max iterations reached.")

### 🧪 Step 4: Testing
Let's run the scenarios.

In [6]:
# SCENARIO 1: The Happy Path (Composite Task)
run_agent("Find the contact for Acme Corp and book a 45 min meeting with them on Oct 20th.")


🚀 STARTING AGENT FOR: 'Find the contact for Acme Corp and book a 45 min meeting with them on Oct 20th.'
--------------------------------------------------
🤖 AI Decided to call: 1 tools
   👉 Invoking 'get_client_details' with args: {'client_name': 'Acme Corp'}
   ✅ Result: Client: Acme Corp | Industry: Retail | Contact: Alice Smith
🤖 AI Decided to call: 1 tools
   👉 Invoking 'schedule_meeting' with args: {'client_name': 'Acme Corp', 'date': 'Oct 20th', 'duration_minutes': 45}
   ✅ Result: SUCCESS: Meeting scheduled with Acme Corp on Oct 20th for 45 minutes.
--------------------------------------------------
🏁 FINAL ANSWER: I have successfully scheduled a 45-minute meeting with Acme Corp on October 20th. The key contact for Acme Corp is Alice Smith.


'I have successfully scheduled a 45-minute meeting with Acme Corp on October 20th. The key contact for Acme Corp is Alice Smith.'

In [7]:
# SCENARIO 2: Data Validation (Implicit Integer Conversion)
# Note: The LLM should automatically convert '30 minutes' to the integer 30 because of Pydantic.
run_agent("Book a meeting with Globex for 30 minutes on Friday.")


🚀 STARTING AGENT FOR: 'Book a meeting with Globex for 30 minutes on Friday.'
--------------------------------------------------
🤖 AI Decided to call: 1 tools
   👉 Invoking 'get_client_details' with args: {'client_name': 'Globex'}
   ✅ Result: Client: Globex | Industry: Tech | Contact: Bob Jones
🤖 AI Decided to call: 1 tools
   👉 Invoking 'schedule_meeting' with args: {'client_name': 'Globex', 'date': 'Friday', 'duration_minutes': 30}
   ✅ Result: SUCCESS: Meeting scheduled with Globex on Friday for 30 minutes.
--------------------------------------------------
🏁 FINAL ANSWER: The meeting with Globex has been successfully scheduled for 30 minutes on Friday.


'The meeting with Globex has been successfully scheduled for 30 minutes on Friday.'

In [8]:
# SCENARIO 3: Error Handling (Client not found)
# The agent should NOT crash. It should apologize.
run_agent("Who is the contact for Nvidia?")


🚀 STARTING AGENT FOR: 'Who is the contact for Nvidia?'
--------------------------------------------------
🤖 AI Decided to call: 1 tools
   👉 Invoking 'get_client_details' with args: {'client_name': 'Nvidia'}
   ✅ Result: Error: Client 'Nvidia' not found in the database.
--------------------------------------------------
🏁 FINAL ANSWER: I'm sorry, but I couldn't find any contact information for Nvidia in the database. If you have any other questions or need further assistance, feel free to ask!


"I'm sorry, but I couldn't find any contact information for Nvidia in the database. If you have any other questions or need further assistance, feel free to ask!"